In [33]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [34]:
spotify = pd.read_csv('decades.csv')
spotify.head()

,Unnamed: 0,url,WeekID,Week Position,Song_x,Performer_x,SongID,Instance,Previous Week Position,Peak Position,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,Decade
0,1,http://www.billboard.com/charts/hot-100/2014-0...,3/22/2014,45,Sail,AWOLNATION,SailAWOLNATION,2,41.0,17,...,1.0,0.0568,0.452,0.609,0.0953,0.243,119.038,4.0,2014,2010s
1,2,http://www.billboard.com/charts/hot-100/2009-1...,10/10/2009,48,I'm Yours,Jason Mraz,I'm YoursJason Mraz,1,49.0,6,...,1.0,0.0468,0.595,0.000,0.1050,0.718,150.953,4.0,2009,2000s
2,3,http://www.billboard.com/charts/hot-100/1998-1...,10/10/1998,45,How Do I Live,LeAnn Rimes,How Do I LiveLeAnn Rimes,1,48.0,2,...,1.0,0.0279,0.128,0.000,0.0822,0.258,128.303,4.0,1998,1990s
3,6,http://www.billboard.com/charts/hot-100/2012-0...,4/14/2012,49,Rolling In The Deep,Adele,Rolling In The DeepAdele,2,43.0,1,...,1.0,0.0294,0.131,0.000,0.0527,0.522,104.945,4.0,2012,2010s
4,8,http://www.billboard.com/charts/hot-100/2007-1...,12/1/2007,47,Before He Cheats,Carrie Underwood,Before He CheatsCarrie Underwood,1,43.0,8,...,0.0,0.0405,0.271,0.000,0.1190,0.290,147.905,4.0,2007,2000s


In [35]:
#Turning the column into categorical from numeric and changing to binary: Number1 and NotNumber1
spotify['Peak Position'].mask(spotify['Peak Position'] == 1, 'Number1', inplace=True)

In [36]:
spotify['Peak Position'].mask(spotify['Peak Position'] != 'Number1', 'NotNumber1', inplace=True)

In [51]:
#Dropping columns we don't need for the model
dropped = spotify.drop(["Weeks on Chart","year", "WeekID", "Unnamed: 0", "url", "spotify_track_id", "spotify_track_explicit", "Week Position", "Song_x", "Performer_x", "Instance", "Previous Week Position", "Performer_y", "Song_y", "spotify_genre", "spotify_track_preview_url","spotify_track_album", "spotify_track_duration_ms", "Decade", "SongID" ], axis=1)

In [52]:
#Second drop of columns that had a low coeff_ score compared to other variables
dropped2 =dropped.drop(["key", "loudness", "mode", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "time_signature"], axis=1)

In [53]:
dropped2

,Peak Position,spotify_track_popularity,danceability,energy,speechiness
0,NotNumber1,74.0,0.825,0.435,0.0568
1,NotNumber1,83.0,0.686,0.457,0.0468
2,NotNumber1,68.0,0.577,0.462,0.0279
3,Number1,76.0,0.729,0.756,0.0294
4,NotNumber1,74.0,0.519,0.749,0.0405
...,...,...,...,...,...
10713,NotNumber1,64.0,0.884,0.814,0.3030
10714,NotNumber1,74.0,0.588,0.567,0.0628
10715,NotNumber1,71.0,0.643,0.503,0.2160
10716,NotNumber1,54.0,0.701,0.915,0.1280


In [54]:
# Assign X (data) and y (target)
X = dropped2.drop("Peak Position", axis=1)
y = dropped2["Peak Position"]
print(X.shape, y.shape)

(10718, 4) (10718,)


In [55]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

#This stratify parameter makes a split so that the proportion of values in the sample produced will be the same as
#the proportion of values provided to parameter stratify.
#For example, if variable y is a binary categorical variable with values 0 and 1 and there are 25% of zeros and 75% 
#of ones, stratify=y will make sure that your random split has 25% of 0's and 75% of 1's.

In [56]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [57]:
classifier.fit(X_train, y_train)

/Users/davidlopez/anaconda3/envs/PythonWebMongo/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [58]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.9548395123164967
Testing Data Score: 0.9548507462686567


In [59]:
predictions = classifier.predict(X_test)
print(f"First 1000 Predictions:   {predictions[:100]}")
print(f"First 1000 Actual labels: {y_test[:100].tolist()}")

First 1000 Predictions:   ['NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1'
 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1'
 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1'
 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1'
 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1'
 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1'
 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1'
 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1'
 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1'
 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1'
 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1'
 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1'
 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1'
 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber1'
 'NotNumber1' 'NotNumber1' 'NotNumber1' 'NotNumber

In [60]:
DataFrame = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

In [61]:
DataFrame.sort_values(by = 'Actual', ascending=False)

,Prediction,Actual
686,NotNumber1,Number1
143,NotNumber1,Number1
949,NotNumber1,Number1
943,NotNumber1,Number1
823,NotNumber1,Number1
...,...,...
918,NotNumber1,NotNumber1
919,NotNumber1,NotNumber1
920,NotNumber1,NotNumber1
921,NotNumber1,NotNumber1


In [62]:
#These coefficients are in scientific notation 1.2e+01 would be 12, 1.2e+00 would be 1.2, 1.2e-01 would be 0.12
classifier.coef_

array([[ 0.05687111,  0.1635753 , -1.80301587, -1.73977435]])

In [63]:
DataFrame["Prediction"].value_counts()

NotNumber1    2680
Name: Prediction, dtype: int64

In [64]:
DataFrame["Actual"].value_counts()

NotNumber1    2559
Number1        121
Name: Actual, dtype: int64